In [ ]:
# 유사도 관련 패키지
# !pip install fasttext

In [ ]:
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# db 연동
import pymysql

# 주식 데이터
import FinanceDataReader as fdr

# nlp
from konlpy.tag import Okt

# 유사도 계산
from gensim import models

In [ ]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()
    
    # ============== 날짜 조정
    ## 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
    for i in range(len(df)):
        if df['date_h'].iloc[i] in after_market:
            df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
    return df_1

In [ ]:
def giro_dic(df,num = 0):
    
    df = set_data(df, num)
    okt = Okt()
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')
    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = wordList
    
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
    media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
            if word in media:
                df['Contents_input'][i].remove(word) 
    
    # 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    
    # 주식 감성 사전 로딩 : koself
    koself_pro = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/KOSELF_pos.txt'
    koself_neg = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/KOSELF_neg.txt'
    
    positive = list(pd.read_csv(koself_pro, header=None)[0])
    negative = list(pd.read_csv(koself_neg, header=None)[0])
    
    # koself 만 쓴 감성점수
    df['koself'] = 0
    for index, wordList in enumerate(df['Contents_input']):
        for word in wordList:
            if word in positive:
                df['koself'][index] += 1
            if word in negative:
                df['koself'][index] -= 1
                
    # 사전 구축
    # 유사도 계산으로 긍부정 단어 추가
    # 유사도 계산으로 애매한 단어 긍부정 판단--> 긍부정 단어에 다포함되어있는
    # 사용 모델 -->  fastText의 이미 학습된 한국어 모델 사용
    try:
        print(ko_model, '모델 로딩 안해도 됨')
    except:
        ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')
    
    # 단어 수를 늘리기 위해서 유사도 50%이상인 단어를 추가
    for i in positive:
        pos_si = [j[0] for j in ko_model.wv.similar_by_word(i, 10) if j[1] >= 0.5]
    for i in negative:
        neg_si = [j[0] for j in ko_model.wv.similar_by_word(i, 10) if j[1] >= 0.5]
    
    pos_gi = positive + pos_si
    neg_gi = negative + neg_si
    
    
    for index, ud in enumerate(df['ud']):
        if ud == 1: # positive?
            for i in df['Contents_input'][index]:
                if (i in pos_gi) and (i in neg_gi):
                    # 10개의 유사한 단어에서 positive가 5 이상이면 positive로 판단
                    S_word = [j[0] for j in ko_model.wv.similar_by_word(i, 10)]
                    word_num = 0
                    for word in S_word:
                        if word in pos_gi:
                            word_num += 1
                        if word_num >= 5:
                            pos_gi.append(i)
                            break
                else:
                    pos_gi.append(i)
                    
        elif ud == -1:
            for i in df['Contents_input'][index]:
                if (i in pos_gi) and (i in neg_gi):
                    # 10개의 유사한 단어에서 negative가 5 이상이면 negative로 판단
                    S_word = [j[0] for j in ko_model.wv.similar_by_word(i, 10)]
                    word_num = 0
                    for word in S_word:
                        if word in neg_gi:
                            word_num += 1
                        if word_num >= 5:
                            neg_gi.append(i)
                            break
                else:
                    neg_gi.append(i)      
        else:
            continue
    
    # 사전 중복 제거
    pos_gi = list(set(pos_gi))
    neg_gi = list(set(neg_gi))
    
    # 만들어진 사전으로 점수 도출
    df['giro'] = 0
    for index, wordList in enumerate(df['Contents_input']):
        for word in wordList:
            if word in pos_gi:
                df['giro'][index] += 1
            if word in neg_gi:
                df['giro'][index] -= 1
            
    return df, pos_gi, neg_gi

In [ ]:
df = pd.read_csv('./data/전처리 전 데이터/df_매일경제_삼성전자.csv')

In [ ]:
import os
# 모델 크기가 크기 때문에 다른 곳에 저장__git에 커밋되지 않게
print(os.getcwd()) # 현재 디렉토리 위치

In [ ]:
os.chdir("../.git/비공개")

In [ ]:
print(os.getcwd())

In [ ]:
# 페이스북 한국어 임베딩 모델 다운 --> 1번만 하면 됨
import fasttext
import fasttext.util
fasttext.util.download_model('ko', if_exists='ignore')

In [ ]:
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [ ]:
df.head()

In [ ]:
df['title'][0]

In [ ]:
prt=df[:-10]

In [ ]:
pred, pos_gi, neg_gi = giro_dic(prt)
pred.head()

In [ ]:
os.chdir("C:/Users/j/stock_predict/감성분석")

In [ ]:
# 결과 저장

pred.to_csv('pred.csv', index=False)

with open('./pos.txt','w') as f:
    f.write(pos_gi)
with open('./neg.txt', 'w') as f:
    f.write(neg_gi)

#### train --> ud 정확도 판단

In [ ]:
# 구축된 사전 단어 수
len(pos_gi), len(neg_gi)

In [ ]:
(len(pred[(pred.koself > 0) & (pred.ud > 0)])+len(pred[(pred.koself < 0) & (pred.ud < 0)])) / (len(pred[pred.ud > 0])+len(pred[pred.ud < 0]))

In [ ]:
(len(pred[(pred.giro > 0) & (pred.ud >0)]) + len(pred[(pred.giro < 0) & (pred.ud <0)])) / (len(pred[pred.ud > 0])+len(pred[pred.ud < 0]))

#### test --> 정확도 판단

In [ ]:
test = df[-10:]

In [ ]:
test['giro'] = 0
for index, wordList in enumerate(df['Contents_input']):
    for word in wordList:
        if word in pos_gi:
            test['giro'][index] += 1
        if word in neg_gi:
            test['giro'][index] -= 1

In [ ]:
len(test[(test.giro > 0) & (test.ud > 0)])/len(test[test.ud > 0])

## ---------------- db----------------

In [ ]:
import pymysql
import pandas

db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='3.35.70.166', 
    db='proj', 
    charset = 'utf8'
)

cursor = db.cursor(pymysql.cursors.DictCursor)

sql = "select * from news_craw_005930 where length(date) = 10"  # date 널값 제외
cursor.execute(sql)

result = cursor.fetchall()

# 데이터 프레임으로 변경
df = pandas.DataFrame(result)

### ------------- ver2 실습 -----------------

## --------------------실습 ----------------------

## 함수 실습 _ ver1

In [ ]:
df_a, pos_cnt, nega_cnt, positive_r, negative_r =  giro_dic(df, 5930)

In [ ]:
df_a

## 실습 1

In [ ]:
df.head(1)

In [ ]:
df['text'].str.replace('[\n|\t]','')

In [ ]:
# pip install xlrd
# !pip install openpyxl

In [ ]:
df_hundai = pd.read_excel('./news_craw_hyundai.xlsx', engine = 'openpyxl')

In [ ]:
df_hundai = df_hundai.dropna()

In [ ]:
df_hundai['date'] = df_hundai['date'].astype(int)

In [ ]:
df_hundai.head()

In [ ]:
df_hundai.drop('Unnamed: 0',1, inplace= True)

In [ ]:
df_hundai.columns = ['st_n', 'st_cd', 'news', 'n_date', 'title', 'url', 'text']

In [ ]:
df_hundai.head(1)

In [ ]:
df_a.head()

In [ ]:
pos_cnt

In [ ]:
nega_cnt

In [ ]:
positive_r

In [ ]:
negative_r